In [27]:
# import numpy as np
# import pandas as pd

In [28]:
import openai
import os

In [29]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [30]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")



In [31]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [32]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [33]:
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

from langchain_community.document_loaders import PyPDFLoader

file_path = "/home/nouman-aziz/Downloads/Grid Needs Assessment SCE 2024.pdf"

loader = PyPDFLoader(file_path)

pages = []
async for page in loader.alazy_load():
    pages.append(page)

docs = loader.load()

In [34]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

/home/nouman-aziz/demo_bot/venv/lib/python3.12/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [35]:
for i in range(10):
    user_input = input("Enter your question here: ")
    if user_input.lower() =="quit":
       break
    response = graph.invoke({"question": user_input})
    print(response["answer"])

The document contains various attachments related to grid assessment and energy needs, including datasets and payment information for distributed energy resources (DER). It appears to provide insight into the energy sector's requirements and evaluations. The specific details of each attachment may vary, focusing on different aspects of energy management.
